In [2]:
#Data Reading

#Import important libraries to read and process the data
import numpy as np
import pandas as pd

#Read the .dat file
seqInput = pd.read_table('SequencingData 300718.dat')

#Save the file in a .csv file to make its inspection more easier using a tab delimeter 
seqInput.to_csv('seqInputToRead.csv', sep='\t')



In [3]:
#Question 1:
#Isolate the important field: the predictions and the observations
predictionSeqInput = seqInput.iloc[4:8,3:73]
observedSeqInput = seqInput.iloc[9:13,3:73]

#Take the values from the dataframe
arrayPredictionSeqInput = predictionSeqInput.values
arrayObservedSeqInput = observedSeqInput.values

#Defintion of important functions
def MinMaxNormalization(inputSequence):
    #Normalizes an input sequence along the cycles to the range 1.0-0.0
    outputSequence = np.zeros(inputSequence.shape)
    for i in range(inputSequence.shape[1]):
        vector = inputSequence[:,i]
        vector2 = pd.to_numeric(vector)
        minValue = min(vector2).astype(float)
        maxValue = max(vector2).astype(float)
        outputSequence[:,i] = (vector2.astype(float)-minValue)/(maxValue-minValue)
    return outputSequence

def reduceSequence(inputSequence):    
    #Finds the index location of the maximum per cycle
    index = np.argmax(inputSequence, axis=0)
    return index

def computeHammingDistance(inputSequence_1, inputSequence_2):
    #Computes the Hamming distane between two input sequences
    return sum(el1 != el2 for el1, el2 in zip(inputSequence_1, inputSequence_2)) 

#Normalize the sequences with respect to the cycles
normArrayObservedSeqInput = MinMaxNormalization(arrayObservedSeqInput)
normArrayPredictionSeqInput = MinMaxNormalization(arrayPredictionSeqInput)

#Locate which nucleotide gave the maximum intensity per cycle
cleanArrayObservedSeqInput = reduceSequence(normArrayObservedSeqInput)
cleanArrayPredictionSeqInput = reduceSequence(normArrayPredictionSeqInput)

#Compute the Hamming distance between the cleaned observed and predicted sequences
hammingDist = computeHammingDistance(cleanArrayObservedSeqInput, cleanArrayPredictionSeqInput)

In [4]:
hammingDist

0

In [5]:
#Question 3:
#Sort the normalized intensity of the observation with respect to the cycle
sortedNormArrayObservedSeqInput = np.sort(normArrayObservedSeqInput, axis=0)

#Compute the correlation between the highest and the second highest intensity per cycle
corrSecondVsFirst = np.correlate(sortedNormArrayObservedSeqInput[2,:], sortedNormArrayObservedSeqInput[3,:])/70
print(corrSecondVsFirst)

[0.17139728]


In [6]:
#Compute the correlation between the highest and the third highest intensity per cycle
corrThirdVsFirst = np.correlate(sortedNormArrayObservedSeqInput[1,:], sortedNormArrayObservedSeqInput[3,:])/70
print(corrThirdVsFirst)

[0.05077559]


In [7]:
#Compute the correlation between the highest and the fourth highest intensity per cycle
corrFourthVsFirst = np.correlate(sortedNormArrayObservedSeqInput[0,:], sortedNormArrayObservedSeqInput[3,:])/70
print(corrFourthVsFirst)

[0.]


In [8]:
def forwardDiff(inputSequence):
    #Compute the finite difference (gradient) to measure when we have transitions in the observations
    outputSequence = np.zeros(len(inputSequence)-1)
    for i in range(len(inputSequence)-1):
        val1 = inputSequence[i]
        val2 = inputSequence[i+1]
        outputSequence[i] = val2-val1
    return outputSequence

#Find the locations where we have transitions in the observation of a nucleotide per cycle
diffSequence = forwardDiff(cleanArrayObservedSeqInput)
#Append the first entry because it is forward difference
diffSequence = np.append(0,diffSequence)

In [9]:
diffSequence

array([ 0., -1., -1.,  1.,  0., -1., -1.,  2.,  0.,  0.,  0.,  0.,  0.,
        0., -1.,  1., -1.,  2.,  0.,  0.,  0., -2.,  0.,  0.,  0.,  2.,
       -2., -1.,  1.,  0.,  0.,  2., -3.,  0.,  1.,  0.,  0.,  2., -2.,
        2., -2.,  0.,  0.,  0., -1.,  2., -2.,  2.,  0.,  0.,  0., -1.,
        1., -2.,  2.,  0.,  0.,  0., -2.,  1.,  0.,  1.,  1., -3.,  3.,
        0., -1.,  1., -1., -1.])

In [10]:
#Find the locations where we have transitions and conservation of the "winning" nucleotide
transitionsBase = abs(diffSequence)>0
conservationBase = ~transitionsBase

#Keep only the transitions or the conservations from the normalized intensities
secondHighestIntensityOnTransitions = sortedNormArrayObservedSeqInput[2,:]*transitionsBase
secondHighestIntensityOnConservation = sortedNormArrayObservedSeqInput[2,:]*conservationBase

#Remove the zeros 
transitionsBases = secondHighestIntensityOnTransitions[secondHighestIntensityOnTransitions != 0]
conservationBases = secondHighestIntensityOnConservation[secondHighestIntensityOnConservation != 0]

In [11]:
#Compute 1st order statistics
finalAnalysisResult = [[np.mean(transitionsBases), np.std(transitionsBases)],[np.mean(conservationBases), np.std(conservationBases)]]
print(finalAnalysisResult)

[[0.25365355065696693, 0.1344846522703113], [0.06172225154975606, 0.039642636818400134]]
